# Quickstart
有两种 API 可供选择：
- [Use the Graph API](https://docs.langchain.com/oss/python/langgraph/quickstart#use-the-graph-api) if you prefer to define your agent **as a graph of nodes and edges.**
- [Use the Functional API](https://docs.langchain.com/oss/python/langgraph/quickstart#use-the-functional-api) if you prefer to define your agent **as a single function.**

接下来用 Graph API 给出一个完整的流程：

1. Define tools and model
2. Define state
3. Define model node
4. Define tool node
5. Define end logic
6. Build and compile the agent

## 1. Define tools and model
这部分主要是 用 LangChain 的 API 来实现。不再赘述。

## 2. Define state
图的 State 是用来 存储 messages 和 the number of LLM calls. 同样参看 LangChain。

````py
    from langchain.messages import AnyMessage
    from typing_extensions import TypedDict, Annotated
    import operator


    class MessagesState(TypedDict):
        messages: Annotated[list[AnyMessage], operator.add]
        llm_calls: int
````

## 3. Define model node
提供模型调用的函数
````py
from langchain.messages import SystemMessage


def llm_call(state: dict):
    """LLM decides whether to call a tool or not"""

    return {
        "messages": [
            model_with_tools.invoke(
                [
                    SystemMessage(
                        content="You are a helpful assistant tasked with performing arithmetic on a set of inputs."
                    )
                ]
                + state["messages"]
            )
        ],
        "llm_calls": state.get('llm_calls', 0) + 1
    }
````

## 4. Define tool node
The tool node is used to call the tools and return the results.

## 5. Define end logic
定义终止，根据模型是否发起 工具调用 来路由到对应节点 或是 END 节点

In [ ]:
from typing import Literal
from langgraph.graph import StateGraph, START, END


def should_continue(state: MessagesState) -> Literal["tool_node", END]:
    """Decide if we should continue the loop or stop based upon whether the LLM made a tool call"""

    messages = state["messages"]
    last_message = messages[-1]

    # If the LLM makes a tool call, then perform an action
    if last_message.tool_calls:
        return "tool_node"

    # Otherwise, we stop (reply to the user)
    return END

## 6. Build and compile the agent
The agent is built using the StateGraph class and compiled using the compile method.代码示例：

In [ ]:
# Build workflow
agent_builder = StateGraph(MessagesState)

# Add nodes
agent_builder.add_node("llm_call", llm_call)
agent_builder.add_node("tool_node", tool_node)

# Add edges to connect nodes
agent_builder.add_edge(START, "llm_call")
agent_builder.add_conditional_edges(
    "llm_call",
    should_continue,
    ["tool_node", END]
)
agent_builder.add_edge("tool_node", "llm_call")

# Compile the agent
agent = agent_builder.compile()

# Show the agent
from IPython.display import Image, display
display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

# Invoke
from langchain.messages import HumanMessage
messages = [HumanMessage(content="Add 3 and 4.")]
messages = agent.invoke({"messages": messages})
for m in messages["messages"]:
    m.pretty_print()